In [1]:
import os
from osgeo import gdal
import pandas as pd
import numpy as np
import pickle
import osgeo.ogr as ogr
import osgeo.osr as osr
import shapefile

In [2]:
ID_point = pd.read_csv("ID+name.csv")
lat_lon = pd.read_csv("lat_lon.csv")
resp = 0.01

In [ ]:
def vector2raster(inputfilePath, outputfile, resp):
    sf = shapefile.Reader(inputfilePath)
    # read the boundaries of shp
    min_x, min_y, max_x, max_y = sf.bbox
    resp = 0.01  # Set the size of each cell
    tifrow = int((max_x - min_x) / resp)
    tifcol = int((max_y - min_y) / resp)

    vector = ogr.Open(inputfilePath)
    layer = vector.GetLayer()

    targetDataset = gdal.GetDriverByName('GTiff').Create(outputfile, tifrow, tifcol, 1, gdal.GDT_Float32)
    transform = (min_x, resp, 0, min_y, 0, resp)

    targetDataset.SetGeoTransform(transform)
    # targetDataset.SetProjection(data.GetProjection())
    band = targetDataset.GetRasterBand(1)
    NoData_value = -999
    band.SetNoDataValue(NoData_value)
    band.FlushCache()
    gdal.RasterizeLayer(targetDataset, [1], layer, )

In [7]:
%%time
for year in range(2001,2021):
    l1 = str(year)
    output = r'Out_tif' + '/' + l1
    
    if year ==2004 or year ==2008 or year == 2012 or year == 2016 or year == 2020:
        day = 366
    else:
        day = 365
    input1 = r'pkl'+ '/'+ l1
    lists = os.listdir(input1)
    lists.sort(key=lambda x: float(x[14:-4]))
    for k in range(0,day):
        pkl_data = pd.read_pickle(input1 + "//" + lists[k])
        shpname = "shp" + '/' + l1 + "/" + lists[k][0:14] + str(k+1)+".shp" 
        data = pd.concat([lat_lon,pkl_data],axis=1)
        data.columns = ["lat","lon","Sm"]
        SpatialReference  = osr.SpatialReference()
        SpatialReference .ImportFromEPSG(4326)
        driver = ogr.GetDriverByName("ESRI Shapefile")
        data_source = driver.CreateDataSource(shpname)
        layer = data_source.CreateLayer("Sm", SpatialReference, ogr.wkbPoint)
        filed_names = list(data)
        for filed_name in filed_names:
            
            #     print(str(data[filed_name].dtypes))
            if("int" in str(data[filed_name].dtypes)):
                field = ogr.FieldDefn(filed_name, ogr.OFTInteger)
                field.SetWidth(10)
            elif("float" in str(data[filed_name].dtypes)):
                field = ogr.FieldDefn(filed_name, ogr.OFTReal)
                field.SetWidth(10)
                field.SetPrecision(5)
            else:
                field = ogr.FieldDefn(filed_name, ogr.OFTString)
                field.SetWidth(10)
            layer.CreateField(field)

        featureDefn = layer.GetLayerDefn()
        feature = ogr.Feature(featureDefn)
        # set geometry
        point = ogr.Geometry(ogr.wkbPoint)
        featureDefn = layer.GetLayerDefn()
        feature = ogr.Feature(featureDefn)
        
        for i in range(len(data)):
            for j in range(len(filed_names)):
                if("int" in str(data[filed_names[j]].dtypes)):
                    feature.SetField(filed_names[j], int(data.iloc[i, j]))
                elif("float" in str(data[filed_names[j]].dtypes)):
                    feature.SetField(filed_names[j], float(data.iloc[i, j]))
                else:
                    feature.SetField(filed_names[j], str(data.iloc[i, j]))
            point.AddPoint(float(data.iloc[i, 1]), float(data.iloc[i, 0]))
            feature.SetGeometry(point)
            layer.CreateFeature(feature)
        data_source.Destroy()

In [6]:
%%time
for year in range(2001,2021):
    l1 = str(year)
    output = r'Out_tif' + '/' + l1
    
    if year ==2004 or year ==2008 or year == 2012 or year == 2016 or year == 2020:
        day = 366
    else:
        day = 365
    input1 = r'pkl'+ '/'+ l1
    lists = os.listdir(input1)
    lists.sort(key=lambda x: float(x[14:-4]))
    for k in range(0,day):
        pkl_data = pd.read_pickle(input1 + "//" + lists[k])
        inputfilePath = "shp" + '/' + l1 + "/" + lists[k][0:14] + str(k+1)+".shp" 
        outputfile = output + '/' + lists[k][0:14] + str(k+1)+".tif" 
        vector2raster(inputfilePath, outputfile, 0.01)